# 데이콘 풍속 예측 모델 만들기  
### 회귀(예측)


In [ ]:
import numpy as np
import pandas as pd

trdf = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/open/train.csv')
tedf = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/open/test.csv')

In [ ]:
trdf.head()

,ID,월,일,측정 시간대,섭씨 온도(°⁣C),절대 온도(K),이슬점 온도(°C),상대 습도 (%),대기압(mbar),포화 증기압(mbar),실제 증기압(mbar),증기압 부족량(mbar),수증기 함량 (g/kg),공기 밀도 (g/m**3),풍향 (deg),풍속 (m/s)
0,TRAIN_00000,7,2,저녁,13.97,287.78,9.84,76.1,992.08,15.98,12.16,3.82,7.66,1198.06,155.60,1.61
1,TRAIN_00001,8,21,오전,16.94,290.85,12.14,73.3,991.07,19.33,14.17,5.16,8.94,1183.67,177.00,1.68
2,TRAIN_00002,11,1,저녁,9.76,283.84,5.40,74.2,988.71,12.10,8.98,3.12,5.67,1213.22,146.20,0.73
3,TRAIN_00003,12,28,오전,5.27,277.30,2.71,83.5,1014.25,8.89,7.43,1.47,4.57,1265.48,264.50,2.71
4,TRAIN_00004,9,26,오후,17.35,290.86,12.68,74.0,995.77,19.84,14.68,5.16,9.22,1187.40,19.34,1.00


In [ ]:
trdf.info() # 결측치 없음, 측정 시간대 라벨 인코딩 필요

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36581 entries, 0 to 36580
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              36581 non-null  object 
 1   월               36581 non-null  int64  
 2   일               36581 non-null  int64  
 3   측정 시간대          36581 non-null  object 
 4   섭씨 온도(°⁣C)      36581 non-null  float64
 5   절대 온도(K)        36581 non-null  float64
 6   이슬점 온도(°C)      36581 non-null  float64
 7   상대 습도 (%)       36581 non-null  float64
 8   대기압(mbar)       36581 non-null  float64
 9   포화 증기압(mbar)    36581 non-null  float64
 10  실제 증기압(mbar)    36581 non-null  float64
 11  증기압 부족량(mbar)   36581 non-null  float64
 12  수증기 함량 (g/kg)   36581 non-null  float64
 13  공기 밀도 (g/m**3)  36581 non-null  float64
 14  풍향 (deg)        36581 non-null  float64
 15  풍속 (m/s)        36581 non-null  float64
dtypes: float64(12), int64(2), object(2)
memory usage: 4.5+ MB


In [ ]:
trdf.describe()

,월,일,섭씨 온도(°⁣C),절대 온도(K),이슬점 온도(°C),상대 습도 (%),대기압(mbar),포화 증기압(mbar),실제 증기압(mbar),증기압 부족량(mbar),수증기 함량 (g/kg),공기 밀도 (g/m**3),풍향 (deg),풍속 (m/s)
count,36581.000000,36581.000000,36581.000000,36581.000000,36581.000000,36581.000000,36581.000000,36581.000000,36581.000000,36581.000000,36581.000000,36581.000000,36581.00000,36581.000000
mean,6.511468,15.648561,9.992694,283.971095,5.453064,75.827418,990.032953,14.037167,9.786895,4.250224,6.178010,1214.580682,176.58240,2.036446
std,3.458678,8.803467,8.237177,8.354235,6.327009,16.089188,8.824477,8.075680,4.185701,5.191128,2.655054,39.528520,84.38269,1.544146
min,1.000000,1.000000,-13.930000,258.870000,-15.560000,23.640000,956.890000,2.070000,1.810000,0.000000,1.120000,1110.650000,0.01000,0.100000
25%,4.000000,8.000000,3.650000,277.700000,0.450000,65.360000,985.100000,7.940000,6.310000,0.950000,3.980000,1184.030000,129.40000,0.920000
50%,7.000000,16.000000,9.130000,283.170000,5.070000,79.000000,990.250000,11.590000,8.770000,2.250000,5.520000,1214.650000,199.50000,1.630000
75%,9.000000,23.000000,16.350000,290.370000,10.950000,89.200000,995.700000,18.620000,13.100000,5.410000,8.270000,1241.600000,233.90000,2.700000
max,12.000000,31.000000,34.340000,308.620000,20.150000,100.000000,1015.300000,54.270000,23.640000,41.230000,14.980000,1349.460000,360.00000,11.890000


In [ ]:
train_x = trdf.drop(columns=['ID', '풍속 (m/s)'], axis = 1)
train_y = trdf['풍속 (m/s)']

In [ ]:
train_y

0        1.61
1        1.68
2        0.73
3        2.71
4        1.00
         ... 
36576    0.77
36577    8.97
36578    2.58
36579    0.36
36580    4.33
Name: 풍속 (m/s), Length: 36581, dtype: float64

ID는 필요없어서 제거

In [ ]:
test_x = tedf.drop(columns=['ID'], axis = 1)
test_y=tedf['ID']

In [ ]:
corr_x = trdf.corr()
corr_x

<ipython-input-11-1f32d4f162a5>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_x = trdf.corr()


,월,일,섭씨 온도(°⁣C),절대 온도(K),이슬점 온도(°C),상대 습도 (%),대기압(mbar),포화 증기압(mbar),실제 증기압(mbar),증기압 부족량(mbar),수증기 함량 (g/kg),공기 밀도 (g/m**3),풍향 (deg),풍속 (m/s)
월,1.000000,-0.004106,0.150799,0.113863,0.225914,0.086025,0.414791,0.138394,0.210962,0.045196,0.204353,-0.028760,0.009146,-0.186495
일,-0.004106,1.000000,0.036927,0.022892,0.034864,-0.021352,0.160538,0.050455,0.051187,0.037222,0.048915,0.011268,0.044461,0.031513
섭씨 온도(°⁣C),0.150799,0.036927,1.000000,0.996372,0.896195,-0.624135,-0.094014,0.962549,0.877030,0.790244,0.876737,-0.958652,0.016658,0.098497
절대 온도(K),0.113863,0.022892,0.996372,1.000000,0.893198,-0.622665,-0.178365,0.956424,0.872587,0.784298,0.873683,-0.979099,0.022562,0.114711
이슬점 온도(°C),0.225914,0.034864,0.896195,0.893198,1.000000,-0.222133,-0.087869,0.812883,0.981239,0.473384,0.980871,-0.875042,0.016848,-0.048942
상대 습도 (%),0.086025,-0.021352,-0.624135,-0.622665,-0.222133,1.000000,0.067025,-0.659667,-0.212495,-0.854891,-0.212696,0.577811,-0.027953,-0.321123
대기압(mbar),0.414791,0.160538,-0.094014,-0.178365,-0.087869,0.067025,1.000000,-0.058629,-0.067328,-0.036927,-0.083381,0.370414,-0.071336,-0.200326
포화 증기압(mbar),0.138394,0.050455,0.962549,0.956424,0.812883,-0.659667,-0.058629,1.000000,0.825225,0.890279,0.824863,-0.904433,-0.012436,0.093631
실제 증기압(mbar),0.210962,0.051187,0.877030,0.872587,0.981239,-0.212495,-0.067328,0.825225,1.000000,0.477460,0.999841,-0.848729,0.004169,-0.073563
증기압 부족량(mbar),0.045196,0.037222,0.790244,0.784298,0.473384,-0.854891,-0.036927,0.890279,0.477460,1.000000,0.477025,-0.722655,-0.022698,0.204976


In [ ]:
train_dr = train_x.drop(columns=['이슬점 온도(°C)', '실제 증기압(mbar)', '수증기 함량 (g/kg)', '풍향 (deg)'], axis=1)
train_dr

,월,일,측정 시간대,섭씨 온도(°⁣C),절대 온도(K),상대 습도 (%),대기압(mbar),포화 증기압(mbar),증기압 부족량(mbar),공기 밀도 (g/m**3)
0,7,2,저녁,13.97,287.78,76.10,992.08,15.98,3.82,1198.06
1,8,21,오전,16.94,290.85,73.30,991.07,19.33,5.16,1183.67
2,11,1,저녁,9.76,283.84,74.20,988.71,12.10,3.12,1213.22
3,12,28,오전,5.27,277.30,83.50,1014.25,8.89,1.47,1265.48
4,9,26,오후,17.35,290.86,74.00,995.77,19.84,5.16,1187.40
...,...,...,...,...,...,...,...,...,...,...
36576,8,10,저녁,14.10,287.79,54.15,993.53,16.11,7.39,1200.85
36577,2,20,오전,3.80,277.91,70.70,988.03,8.02,2.35,1240.06
36578,5,29,저녁,20.06,295.10,81.00,977.90,23.51,4.47,1153.24
36579,9,10,저녁,25.65,299.81,52.81,988.39,32.98,15.56,1144.61


In [ ]:
test_dr = test_x.drop(columns=['이슬점 온도(°C)', '실제 증기압(mbar)', '수증기 함량 (g/kg)', '풍향 (deg)'], axis=1)
test_dr

,월,일,측정 시간대,섭씨 온도(°⁣C),절대 온도(K),상대 습도 (%),대기압(mbar),포화 증기압(mbar),증기압 부족량(mbar),공기 밀도 (g/m**3)
0,3,24,새벽,4.28,278.68,72.5,984.48,8.30,2.28,1233.29
1,9,24,저녁,13.40,286.81,81.8,996.98,15.40,2.80,1206.20
2,5,28,저녁,19.89,294.33,73.2,984.83,23.26,6.23,1163.06
3,1,17,저녁,-2.88,270.44,88.7,998.02,4.94,0.56,1284.19
4,10,22,오후,6.97,281.18,83.4,987.00,10.01,1.66,1223.47
...,...,...,...,...,...,...,...,...,...,...
15673,3,16,새벽,1.61,274.67,85.9,1001.26,6.86,0.97,1266.62
15674,9,5,오후,18.27,292.19,73.0,990.98,21.03,5.68,1177.63
15675,8,8,새벽,16.08,289.66,78.0,994.97,18.30,4.03,1191.84
15676,2,4,새벽,3.07,276.68,74.8,994.31,7.62,1.92,1251.24


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le = le.fit(train_x['측정 시간대'])
train_x['측정 시간대'] = le.transform(train_x['측정 시간대'])
test_x['측정 시간대'] = le.transform(test_x['측정 시간대'])

In [ ]:
train_x

,월,일,측정 시간대,섭씨 온도(°⁣C),절대 온도(K),이슬점 온도(°C),상대 습도 (%),대기압(mbar),포화 증기압(mbar),실제 증기압(mbar),증기압 부족량(mbar),수증기 함량 (g/kg),공기 밀도 (g/m**3),풍향 (deg)
0,7,2,3,13.97,287.78,9.84,76.10,992.08,15.98,12.16,3.82,7.66,1198.06,155.60
1,8,21,1,16.94,290.85,12.14,73.30,991.07,19.33,14.17,5.16,8.94,1183.67,177.00
2,11,1,3,9.76,283.84,5.40,74.20,988.71,12.10,8.98,3.12,5.67,1213.22,146.20
3,12,28,1,5.27,277.30,2.71,83.50,1014.25,8.89,7.43,1.47,4.57,1265.48,264.50
4,9,26,2,17.35,290.86,12.68,74.00,995.77,19.84,14.68,5.16,9.22,1187.40,19.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36576,8,10,3,14.10,287.79,4.99,54.15,993.53,16.11,8.73,7.39,5.48,1200.85,292.90
36577,2,20,1,3.80,277.91,-1.01,70.70,988.03,8.02,5.67,2.35,3.58,1240.06,210.50
36578,5,29,3,20.06,295.10,16.70,81.00,977.90,23.51,19.04,4.47,12.20,1153.24,19.22
36579,9,10,3,25.65,299.81,15.30,52.81,988.39,32.98,17.41,15.56,11.03,1144.61,225.40


In [ ]:
test_y

0        TEST_00000
1        TEST_00001
2        TEST_00002
3        TEST_00003
4        TEST_00004
            ...    
15673    TEST_15673
15674    TEST_15674
15675    TEST_15675
15676    TEST_15676
15677    TEST_15677
Name: ID, Length: 15678, dtype: object

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_clf = DecisionTreeRegressor(random_state=42, max_leaf_nodes = 3, max_features =float)

In [ ]:
dt_clf.fit(train_x , train_y)

DecisionTreeRegressor(random_state=42)

In [ ]:
dt_pred = dt_clf.predict(test_x)
dt_pred

array([2.49, 0.98, 1.7 , ..., 1.14, 1.91, 0.47])

그리드 서치

In [ ]:
from sklearn.model_selection import GridSearchCV
estimator = DecisionTreeRegressor()
param_grid = {'criterion':['mae'], 'max_depth':[None,2,3,4],'max_leaf_nodes':[None,2,3,4],'min_samples_split':[2,3,4], 'min_samples_leaf':[1,2], 'max_features':[None,'sqrt','log2',3,4,5] }
grid = GridSearchCV(estimator, param_grid=param_grid)
grid.fit(train_x, train_y)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
37800 fits failed out of a total of 56700.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18900 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, i

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['mse', 'friedman_mse', 'mae'],
                         'max_depth': [None, 2, 3, 4, 5, 6],
                         'max_features': [None, 'sqrt', 'log2', 3, 4, 5],
                         'max_leaf_nodes': [None, 2, 3, 4, 5, 6, 7],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 3, 4, 5, 6]})

In [ ]:
print(grid.best_score_)
print(grid.best_params_)
df = pd.DataFrame(grid.cv_results_)
print(df)

0.7059485597085787
{'criterion': 'friedman_mse', 'max_depth': None, 'max_features': 5, 'max_leaf_nodes': None, 'min_samples_leaf': 2, 'min_samples_split': 5}
       mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0           0.002814      0.000544              0.0             0.0   
1           0.003272      0.000093              0.0             0.0   
2           0.002429      0.000396              0.0             0.0   
3           0.002126      0.000079              0.0             0.0   
4           0.002159      0.000113              0.0             0.0   
...              ...           ...              ...             ...   
11335       0.002010      0.000017              0.0             0.0   
11336       0.002007      0.000020              0.0             0.0   
11337       0.002041      0.000012              0.0             0.0   
11338       0.002044      0.000062              0.0             0.0   
11339       0.002071      0.000044              0.0          

In [ ]:
estimator = grid.best_estimator_

In [ ]:
y_predict = estimator.predict(test_x)

In [ ]:
y_predict

array([0.92  , 0.975 , 1.74  , ..., 1.26  , 2.0275, 0.45  ])

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/open/sample_submission.csv')

In [ ]:
submission['풍속 (m/s)'] = y_predict
submission.head()

,ID,풍속 (m/s)
0,TEST_00000,0.920000
1,TEST_00001,0.975000
2,TEST_00002,1.740000
3,TEST_00003,1.197500
4,TEST_00004,1.453333


In [ ]:
submission.to_csv('submission.csv', index=  False)

In [ ]:
print(grid.best_params_)

{'criterion': 'friedman_mse', 'max_depth': None, 'max_features': 5, 'max_leaf_nodes': None, 'min_samples_leaf': 2, 'min_samples_split': 5}


# 최종모델  
위에서 그리드서치로 도출된 최적 파라미터와 상관계수로 필요없어 보이는 특성들을 제외한 모델

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_clf = DecisionTreeRegressor(max_depth = None, max_features=5, max_leaf_nodes= None, min_samples_leaf= 2, min_samples_split=5)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le = le.fit(train_dr['측정 시간대'])
train_dr['측정 시간대'] = le.transform(train_dr['측정 시간대'])
test_dr['측정 시간대'] = le.transform(test_dr['측정 시간대'])

In [ ]:
dt_clf.fit(train_dr, train_y)

DecisionTreeRegressor(max_features=5, min_samples_leaf=2, min_samples_split=5)

In [ ]:
last_pred = dt_clf.predict(test_dr)
last_pred

array([1.56      , 1.22      , 2.11      , ..., 1.555     , 1.74333333,
       0.71      ])

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/open/sample_submission.csv')

In [ ]:
submission['풍속 (m/s)'] = last_pred
submission.head()

,ID,풍속 (m/s)
0,TEST_00000,1.56
1,TEST_00001,1.22
2,TEST_00002,2.11
3,TEST_00003,0.98
4,TEST_00004,0.93


In [ ]:
submission.to_csv('submission2.csv', index=False)